In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pandas as pd


In [ ]:
#loading dataset
dt=pd.read_csv("SPAM.csv",encoding = 'Windows-1252')

In [ ]:
import chardet
with open("SPAM.csv",'rb') as rawdata:
  result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}

In [ ]:
dt.head()


,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
dt['SPAM']=dt['type'].map({'spam':1,'ham':0}).astype(int)

In [ ]:
dt.head()


,type,text,SPAM
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
print("COLUMNS IN THE GIVEN DATA")
for col in dt.columns:
  print(col)

COLUMNS IN THE GIVEN DATA
type
text
SPAM


In [ ]:
t=len(dt['type'])
print("NO OF ROWS IN REVIEW COLUMN",t)

NO OF ROWS IN REVIEW COLUMN 116
NO OF ROWS IN LIKED COLUMN 116


In [ ]:
#TOKENIZATION
dt['text'][4]

"Nah I don't think he goes to usf, he lives around here though"

In [ ]:
def tokenizer(text):
  return text.split()

In [ ]:
dt['text']=dt['text'].apply(tokenizer)

In [ ]:
#stemming
dt['text'][4]#before

['Nah',
 'I',
 "don't",
 'think',
 'he',
 'goes',
 'to',
 'usf,',
 'he',
 'lives',
 'around',
 'here',
 'though']

In [ ]:
from nltk.stem.snowball import SnowballStemmer as ss
porter=ss("english",ignore_stopwords=False)

In [ ]:
def stemit(text):
  return [porter.stem(word) for word in text]

In [ ]:
dt['text']=dt['text'].apply(stemit)

In [ ]:
dt['text'][4]

['nah',
 'i',
 "don't",
 'think',
 'he',
 'goe',
 'to',
 'usf,',
 'he',
 'live',
 'around',
 'here',
 'though']

In [ ]:
#LEMMITIZATION
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemmit_it(text):
   return [lemmatizer.lemmatize(word, pos = 'a') for word in text]
dt['text']=dt['text'].apply(lemmit_it)

In [ ]:
dt['text'][4]

['nah',
 'i',
 "don't",
 'think',
 'he',
 'goe',
 'to',
 'usf,',
 'he',
 'live',
 'around',
 'here',
 'though']

In [ ]:
from nltk.corpus import stopwords
stop_words=stopwords.words("english")

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def stop_it(text):
  review = [word for word in text if not word in stop_words]
  return review

In [ ]:
dt['text']=dt['text'].apply(stop_it)

In [ ]:
dt['text'][4]


['nah', 'think', 'goe', 'usf,', 'live', 'around', 'though']

In [ ]:
dt.head()


,type,text,SPAM
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0
4,ham,"[nah, think, goe, usf,, live, around, though]",0


In [ ]:
dt['text']=dt['text'].apply(' '.join)
dt.head()

,type,text,SPAM
0,ham,"go jurong point, crazy.. avail onli bugi n gre...",0
1,ham,ok lar... joke wif u oni...,0
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,1
3,ham,u dun say earli hor... u c alreadi say...,0
4,ham,"nah think goe usf, live around though",0


In [ ]:
#vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
y=dt.SPAM.values
x=tfidf.fit_transform(dt['text'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_text,y_train,y_text=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_text)
from sklearn.metrics import accuracy_score
acc_log = accuracy_score(y_pred, y_text)*100
print("accuracy:",acc_log )

accuracy: 87.5


In [ ]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(random_state=0)
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_text)
acc_linear_svc =accuracy_score(y_pred, y_text) * 100
print("accuracy:",acc_linear_svc)

accuracy: 87.5
